# Load the data

In [ ]:
# Mount Google Drive locally 
# Using the instructions found here https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA&line=1&uniqifier=1

In [ ]:
# Load the pickled dataframe using the "read_pickle()"" function from "pandas"
# df = ...

# Install AIF360

In [ ]:
# install AIF360
!pip install aif360

# Train a classifier, implement post-processing, and compute metrics

In [ ]:
# Instantiate the classifier
from sklearn.linear_model import LogisticRegression
# lr = ...

# instantiate the cross-validation scheme
from sklearn.model_selection import StratifiedKFold
# mv = ...

# setup the performance metrics to be computed
from sklearn import metrics
# perf_metrics = {"Accuracy": ..., 
#                 "Precision": ..., 
#                 "Recall": ...,
#                 "AUC": ..., 
#                 "F1-Score": ..., 
#                 }

# Set up the fairness metrics to be computed using AIF360
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

# privileged_group = ...
# unprivileged_group = ...

In [ ]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing

# Learn parameters to equalize odds and apply to create a new dataset
# epp = EqOddsPostprocessing(...)


In [ ]:
# Train a logistic regression classifier on the dataset and calculate metrics

import pandas as pd
import numpy as np
k, i = True, 1

# instantiating X
# X = ...

# instantiating the target variable
# y = ...

for (train, test) in mv.split(X, y):
    # X_train = ...
    # y_train = ...
    # fit model
    # lr = lr.fit(...)
   
    # get predictions in the test set
    # X_test = ...
    # y_test = ...
    # ypred_prob = lr.predict_proba ... # get probabilities
    # ypred_class = lr.predict ...

    # fit post-processing using results from 60% of the test set
    test_pct = 0.4
    n = int(len(y_test))
    n_2 = int(n* (1-test_pct))
    indices = np.random.permutation(n)
    pp_indices = indices[:n_2]
    test_indices = indices[n_2:]

    pp_dataset = X_test.iloc[pp_indices].copy()
    pp_dataset[df.columns[-1]] = np.expand_dims(y_test.iloc[pp_indices], axis=1)
    # pp_dataset_BLD = BinaryLabelDataset(...)

    pp_pred = X_test.iloc[pp_indices].copy()
    pp_pred[df.columns[-1]] = np.expand_dims(ypred_class[pp_indices], axis=1)
    # pp_pred_BLD = BinaryLabelDataset(...)

    # epp = epp.fit(...)

    # Use epp to post-process predictions on the other 40% of the test set
    actuals_test = X_test.iloc[test_indices].copy()
    actuals_test[df.columns[-1]] = np.expand_dims(y_test.iloc[test_indices], axis=1)
    # actuals_test_BLD = BinaryLabelDataset(...)
    pred_test = X_test.iloc[test_indices].copy()
    pred_test[df.columns[-1]] = np.expand_dims(ypred_class[test_indices], axis=1)
    # pred_test_BLD = BinaryLabelDataset(...)

    # transf_pred_test_BLD = epp.predict(...)

    # compute performance metrics
    metrics = []
    # metric_CM = ClassificationMetric(...)
    for pf in perf_metrics.keys():
        if pf in ["AUC", "Brier"]:
            # metrics += [[pf, ...]]
        else:
            # metrics += [[pf, ...]]
    
    # Compute fairness metrics
    # metrics += [['Statistical Parity Difference', ...]]
    # metrics += [['Disparate Impact', ...]]
    # metrics += [['Equal Opportunity Difference', ...]]
    # metrics += [['Average Odds Difference', ...]]
    # metrics += [['Accuracy Male', ...]]
    # metrics += [['Accuracy Female', ...]]


    # concatenate results
    df_m = pd.DataFrame(metrics, columns=["Metric", "Value"])
    df_m["Fold"] = i
    i += 1
    if k:
        df_metrics = df_m.copy()
        k=0
    else:
        df_metrics = pd.concat([df_metrics, df_m.copy()], axis=0, ignore_index=True)

# Display the metrics in a table

In [ ]:
# Display metrics
# metrics_table = df_metrics.pivot_table(...)
metrics_table

# Chart the metrics

In [ ]:
# Construct a DataFrame with allowable thresholds for plotting

fairness_df = pd.DataFrame(columns=["Metric", "Value", "Min", "Max"])
fairness_df.loc[0] = ['Disparate Impact'] + list((metrics_table[('mean', 'Value')]['Disparate Impact'], 0.8, 1.2))
fairness_df.loc[1] = ['Statistical Parity Difference'] + list((metrics_table[('mean', 'Value')]['Statistical Parity Difference'], -0.1, 0.1))
fairness_df.loc[2] = ['Average Odds Difference'] + list((metrics_table[('mean', 'Value')]['Average Odds Difference'], -0.1, 0.1))
fairness_df.loc[3] = ['Equal Opportunity Difference'] + list((metrics_table[('mean', 'Value')]['Equal Opportunity Difference'], -0.1, 0.1))

In [ ]:
# Chart the metrics

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import numpy as np

def plot_metrics(df_fair): 
  def plot_metric(metric, ylim):
    fig, ax = plt.subplots()
    ax = sns.barplot(data=metric, x="Metric", y="Value")
    plt.axhline(np.mean(metric[["Min", "Max"]].values), color='black')
    plt.ylim(*ylim)
    range_height = metric[["Max"]].values[0][0] - metric[["Min"]].values[0][0]
    ax.add_patch(patches.Rectangle((-1, metric[["Min"]].values[0][0]), 2 , range_height, facecolor="green", alpha=0.3))

  plot_metric(df_fair.iloc[[0]], (0, 2))
  plot_metric(df_fair.iloc[[1]], (-1, 1))
  plot_metric(df_fair.iloc[[2]], (-1, 1))
  plot_metric(df_fair.iloc[[3]], (-1, 1))

plot_metrics(fairness_df)